In [2]:
!pip install nltk

You should consider upgrading via the 'c:\users\gitsa\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached regex-2022.3.15-cp39-cp39-win_amd64.whl (274 kB)


# Locading Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import json
import string
import re
import math
from nltk.stem import PorterStemmer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Preprocessing Function

In [2]:
def preprocess(input):
    input=input.replace('\a',' ')
    input=input.replace('\b',' ')
    input=input.replace('\f',' ')
    input=input.replace('\n',' ')    
    input=input.replace('\r',' ')
    input=input.replace('\t',' ')
    input=input.replace('\v',' ')
    # convert to lower case
    output = input.lower()
    # remove punctuations
    punctuations=string.punctuation.replace("'",'')
    output = "".join([char if char not in punctuations else ' ' for char in output])
    output = output.replace("'",'')
    # tokenize
    output = nltk.word_tokenize(output)
    # removing words with special characters
    output = [word for word in output if re.sub(r'[^\x20-\x7e]','',word) == word]
    # remove stopwords and numeric tokens
    output = [word.strip() for word in output if word not in nltk.corpus.stopwords.words('english') and not word.isnumeric()]
    # stemming
    # output = [PorterStemmer().stem(word) for word in output]
    return output

In [18]:
raw_data = json.load(open("C:\\Users\\gitsa\\PycharmProjects\\class12\\preprocessed.json", "r"))

In [4]:
# preprocessing
for doc in raw_data:
    raw_data[doc] = preprocess(raw_data[doc])

NameError: name 'raw_data' is not defined

# Loading already preprocessed file

In [3]:
raw_data = json.load(open("C:\\Users\\gitsa\\PycharmProjects\\class12\\preprocessed.json", "r"))

# doc to doc-id mapping

In [4]:
def map_docs(raw_data):
    doc_ids = {}
    id = 1
    for doc in raw_data:
        doc_ids[doc] = id
        id += 1
    return doc_ids

doc_ids = map_docs(raw_data)

# creating unigram inverted index


In [5]:
def create_index(doc_ids):
    index = {}
    for doc in doc_ids:
        for token in raw_data[doc]:
            # if token exists in index, add doc id
            if token in index.keys():
                index[token][1].add(doc_ids[doc])
                index[token][0] = len(index[token][1])
            # if token does not exist in index, add to index
            else:
                index[token] = [1, {doc_ids[doc]}]
    return index

index = create_index(doc_ids)

# Finding idf values of words

In [6]:
idfValue={}
totalLen=len(doc_ids)
for word in index:
    idfValue[word] = math.log2(totalLen/(index[word][0]+1))

### Storing terms of a doc in dictionary manner for easier access, storing doc length

In [7]:
doc_terms={}
doc_len={}
ctr=0
for doc in raw_data:
    ctr+=1
    terms=raw_data[doc]
    setterms=set(terms)
    temp_dict={}
    for word in setterms:
        temp_dict[word]=terms.count(word)
    # print(ctr,len(raw_data[doc]))
    doc_terms[doc_ids[doc]]=temp_dict
    doc_len[doc_ids[doc]]=len(terms)

### Storing max frequency in a doc for double normalization

In [8]:
doc_max={}
for doc in doc_terms:
    maxa=0
    for word in doc_terms[doc]:
        maxa=max(maxa,doc_terms[doc][word])
    doc_max[doc]=maxa

In [9]:
print(doc_max)

{1: 6, 2: 66, 3: 91, 4: 20, 5: 12, 6: 13, 7: 38, 8: 8, 9: 6, 10: 132, 11: 24, 12: 4, 13: 9, 14: 5, 15: 16, 16: 11, 17: 8, 18: 10, 19: 56, 20: 5, 21: 3, 22: 72, 23: 9, 24: 4, 25: 16, 26: 4, 27: 6, 28: 147, 29: 24, 30: 91, 31: 74, 32: 29, 33: 8, 34: 6, 35: 14, 36: 53, 37: 27, 38: 27, 39: 10, 40: 8, 41: 4, 42: 9, 43: 30, 44: 6, 45: 11, 46: 6, 47: 81, 48: 4, 49: 6, 50: 18, 51: 12, 52: 7, 53: 15, 54: 41, 55: 50, 56: 33, 57: 66, 58: 13, 59: 10, 60: 16, 61: 20, 62: 12, 63: 5, 64: 8, 65: 5, 66: 60, 67: 9, 68: 127, 69: 19, 70: 27, 71: 12, 72: 4, 73: 15, 74: 9, 75: 20, 76: 4, 77: 3, 78: 6, 79: 8, 80: 5, 81: 4, 82: 15, 83: 16, 84: 201, 85: 54, 86: 4, 87: 9, 88: 108, 89: 194, 90: 10, 91: 8, 92: 209, 93: 44, 94: 24, 95: 12, 96: 21, 97: 53, 98: 24, 99: 9, 100: 26, 101: 26, 102: 11, 103: 6, 104: 6, 105: 17, 106: 24, 107: 19, 108: 3, 109: 9, 110: 26, 111: 4, 112: 5, 113: 7, 114: 126, 115: 18, 116: 12, 117: 27, 118: 89, 119: 4, 120: 17, 121: 27, 122: 3, 123: 6, 124: 7, 125: 6, 126: 15, 127: 11, 128: 48

### Finding tf-idf values by iterating over every word for all docs

In [10]:
def tfvalue(docs, idf, doc_len,doc_max):
    bin = {}
    for doc in docs:
        bin[doc] = {}
    ctr = 0
    for word in idf:
        idfval = idf[word]
        for doc in docs:
            if word in docs[doc].keys():
                val = docs[doc][word]
                bin[doc][word] = [idfval, val * idfval, val * idfval / doc_len[doc], math.log2(1 + val) * idfval,
                                  idfval*(float((0.5+0.5*(val)/doc_max[doc])))]
            else:
                bin[doc][word] = [0, 0, 0, 0, float(0.5*idfval)]
        ctr += 1
        if ctr % 1000 == 0:
            print(ctr)
    return bin

In [11]:
tfidf =tfvalue(doc_terms,idfValue,doc_len,doc_max)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [12]:
for i in range(1,1134):
    print(tfidf[i]['first'][4])

0.6082083127072091
0.4561562345304068
0.5914992931273407
0.5017718579834476
0.4941692540746074
0.526334116765854
0.49216856883543897
0.6272148224793094
0.4561562345304068
0.48380206692618904
0.5131757638467077
0.4561562345304068
0.4561562345304068
0.5473874814364882
0.48466599918855724
0.49762498312408016
0.4561562345304068
0.5017718579834476
0.5294670579370794
0.5473874814364882
0.7602603908840113
0.4624917377877736
0.557524286648275
0.6842343517956102
0.4561562345304068
0.5701952931630085
0.6842343517956102
0.4716717527117132
0.6082083127072091
0.4962578815220909
0.4931418751680074
0.4876152851876762
0.5701952931630085
0.532182273618808
0.4561562345304068
0.49918984156157725
0.4561562345304068
0.4561562345304068
0.5473874814364882
0.4561562345304068
0.5701952931630085
0.4561562345304068
0.4561562345304068
0.4561562345304068
0.5390937317177535
0.532182273618808
0.4843140267853702
0.4561562345304068
0.4561562345304068
0.48149824755987386
0.4561562345304068
0.4561562345304068
0.51697706

### Functions to take input a query, preprocess it, and find its tf-idf value and rank documents

In [13]:
def scoring(query,doc,tfidf,j):
    val=0
    for word in query:
        val+=tfidf[doc][word][j]
    return val

In [14]:
def process(input,j):
    global tfidf,doc_ids
    terms = preprocess(input)
    query = set(terms)

    best_docs = []
    for doc in doc_ids:
        coeff = scoring(query, doc_ids[doc],tfidf,j)
        best_docs.append([coeff, doc])
    # finding 5 best documents
    best_docs.sort(reverse = True)
    return best_docs[:5]

In [15]:

format={0:'Binary Scheme',1:'Raw count Scheme',2:'Term frequency Scheme',3:'Log normalization Scheme',4:'Double normalization Scheme'}
# input and output
def run():
    n=int(input('Enter number of queries:'))
    for i in range(n):
        query = input("Input query: ")
        for j in range(5):
            result = process(query,j)
            print('\nFor',format[j],j)
            print(result)
            print('Top 5 documents:')
            for r in result:
                print(r[1])

In [17]:
run()

Enter number of queries:5
Input query: has saucepan fried

For Binary Scheme 0
[[10.440115250224974, 'vegan.rcp'], [10.440115250224974, 'oculis.rcp'], [10.440115250224974, 'appetiz.rcp'], [5.338577223762911, 'yogurt.asc'], [5.338577223762911, 'woods.txt']]
Top 5 documents:
vegan.rcp
oculis.rcp
appetiz.rcp
yogurt.asc
woods.txt

For Raw count Scheme 1
[[859.5109330258288, 'candy.txt'], [47.099038224662806, 'appetiz.rcp'], [46.86199902736196, 'vegan.rcp'], [37.37004056634038, 'chili.txt'], [16.015731671288734, 'bread.rcp']]
Top 5 documents:
candy.txt
appetiz.rcp
vegan.rcp
chili.txt
bread.rcp

For Term frequency Scheme 2
[[0.09810650050888582, 'johann'], [0.07968025707108822, 'montoys.txt'], [0.07214293545625555, 'caramels.des'], [0.0651046002897916, 'beershrm.fis'], [0.059317524708476795, 'btscke03.des']]
Top 5 documents:
johann
montoys.txt
caramels.des
beershrm.fis
btscke03.des

For Log normalization Scheme 3
[[39.184356051235994, 'candy.txt'], [25.645426401409672, 'appetiz.rcp'], [25.58